## Part 5 Decomposition pathways

Supervisor: Prof. Aschauer

Assistants: Dr. Xing Wang

Room:	N216


### 1. Introduction

We now calculate the barrier for NH$_3$ dissociation. Before the real calculation, We will consider a simple example of the barrier for N$2$ dissociation. https://wiki.fysik.dtu.dk/ase/tutorials/dissociation.html


### 2. N$_2$ dissociation on Cu (111) surface
initial and final states

In [10]:
from ase import Atoms
from ase.build import molecule
from ase.build import fcc111, add_adsorbate
from x3dase.visualize import view_x3d_n
from ase.constraints import FixAtoms

# Find the initial and final states for the reaction.
# Set up a (4 x 4) two layer slab of Cu:
slab = fcc111('Cu',size=(2,2,2))
slab.pbc = [True, True, True]
mask = [atom.symbol == 'Cu' for atom in slab]
slab.set_constraint(FixAtoms(mask=mask))


# Initial state.
# Add the N2 molecule oriented at 60 degrees:
initial = slab.copy()
N2 = molecule('N2')
N2.rotate(90, 'x')
N2.rotate(-60, 'z')
add_adsorbate(initial,N2,height=1.2,position='fcc')
view_x3d_n(initial, bond = 1.0, output = 'htmls/cu-111-n2-initial.html')



In [13]:
# Now the final state.
# Move the second N atom to a neighboring hollow site:
final = slab.copy()
N = Atoms('N')
add_adsorbate(final, N, height=1.2, position='fcc')
add_adsorbate(final, N, height=1.2, position='hcp')
view_x3d_n(final, bond = 1.0, output = 'htmls/cu-111-n2-final.html')


#### relax calculation

In [ ]:
from xespresso import Espresso
from xespresso.tools import mypool, fix_layers, dipole_correction

jobs = {
'initial': initial,
'final': final,
}
queue = {'ntasks': 10, 'partition': 'debug', 'time': '00:10:00'}
print('States    Energy (ev)')
for job, atoms in jobs.items():
    calc = Espresso(label = 'surface/cu-111-n2/{0}/{0}'.format(job),
                    pseudopotentials = mypseudo,
                    calculation = 'relax',
                    ecutwfc = 30,
                    kpts = (1, 1, 1),
                    queue = queue,
                   )
    atoms.calc = calc
    atoms.get_potential_energy()
    calc.read_results()
    jobs[job] = calc.results['atoms']
    e = calc.results['energy']
    print('{0:10s}  {1:10.3f}'.format(job, e))


#### NEB calculation

In [ ]:

from xespresso.neb import NEBEspresso

images = [jobs['initial'], jobs['final']]
path_data = {
'string_method' : 'neb',
'nstep_path': 50,
'opt_scheme': "broyden",
'num_of_images' : 7,
'k_max' : 0.3E0,
'k_min' : 0.2E0,
'CI_scheme' : "auto",
'path_thr': 0.1E0,
}
queue = {'nodes': 2, 'ntasks-per-node': 20, 'partition': 'debug', 'time': '00:10:00'}
calc = NEBEspresso(pseudopotentials = pseudopotentials, 
                 label  = 'molecule/neb/nh3/images',
                 package = 'neb',
                 queue = queue,
                 parallel = '-ni 5',
                 images = images,
                 path_data = path_data,
                 ecutwfc = 30,
                 kpts = (1, 1, 1),
                  )
#
calc.calculate()
calc.read_results()
calc.plot()


### 3. NH$_3$ dissociation
Please read the inital and final structure from your previous calculations, and do the NEB calculation. Please discuss with me when you are doing this.